In [1]:
import torch
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules
import os

# ======================================================================
#     SKRIP FINE-TUNING (TAHAP 2): VERSI EKSPERIMEN LANJUTAN
# ======================================================================

# ===== Step 1: Registrasi & Cek GPU =====
register_all_modules()
print(f"CUDA tersedia? {torch.cuda.is_available()} — {torch.cuda.get_device_name(0)}")

CUDA tersedia? True — NVIDIA GeForce RTX 5060


In [ ]:
# ===== Step 2: Load Config (KONSISTEN dengan Tahap 1) =====
print("--- INFO: Menggunakan model RetinaNet untuk fine-tuning klasifikasi ---")
#untuk retinanet
cfg_path = 'mmdetection/configs/swin/retinanet_swin-t-p4-w7_fpn_1x_coco.py' 
#untuk faster-rcnn
# cfg_path = 'mmdetection/configs/swin/faster-rcnn_swin-t-p4-w7_fpn_1x_coco.py'
cfg = Config.fromfile(cfg_path)

# Deteksi nama model dari path
if "retinanet" in cfg_path.lower():
    model_name = "retinanet"
elif "faster-rcnn" in cfg_path.lower():
    model_name = "fasterrcnn"
else:
    model_name = "custommodel"

print(f"--- INFO: Model yang digunakan: {model_name} ---")

# ===== [PENINGKATAN 1] Freeze Sebagian Backbone =====
# Membekukan 3 dari 4 stage pertama dari Swin Transformer.
# Hanya stage terakhir dan head yang akan dilatih secara intensif.
# Ini membantu mengurangi overfitting dan menstabilkan fine-tuning.
cfg.model.backbone.frozen_stages = 3
print("--- INFO: Membekukan 3 stage pertama dari backbone Swin Transformer. ---")

--- INFO: Menggunakan model RetinaNet untuk fine-tuning klasifikasi ---
--- INFO: Membekukan 3 stage pertama dari backbone Swin Transformer. ---


In [ ]:
# ===== Step 3: Ganti Dataset ke Ornamental Fish & Atur Metainfo =====
data_root = 'dataset_final/'

# Definisikan nama kelas SESUAI URUTAN ID di file JSON Anda
# CLASSES = (
#     'Fish', 'Angel Fish', 'Betta Candy Koi Fighting Fish', 'Candy Koi Betta', 
#     'Galaxy Betta', 'Galaxy Betta Fish', 'Gemrin Koi', 'Guppy Dumbo', 
#     'Guppy Dumbo Fish', 'Juvenile Jewel', 'Juvenile Jewel Fish', 'Koi Gemrin', 
#     'Marble Molly', 'Marble Molly Fish', 'Polar Blue Parrot', 'Rancho', 
#     'Tiger Barb'
# )
# dua belas(12) jenis ikan
CLASSES = (
    'Fish','molly', 'angelfish', 'guppy', 'betta', 'cichlid',
    'gourami', 'carp', 'goldfish', 'discus',
    'pleco', 'barb', 'koi'
)
NUM_CLASSES = len(CLASSES)
metainfo = {'classes': CLASSES}
print(f"--- INFO: Melakukan fine-tuning untuk {NUM_CLASSES} kelas. ---")

# ===== [PENINGKATAN 2] Tambahkan Augmentasi Data =====
train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='Resize', scale=(640, 640), keep_ratio=True),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'), # Menambah variasi warna & kecerahan
    dict(type='PackDetInputs'),
]
val_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(640, 640), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='PackDetInputs', meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor'))
]
print("--- INFO: Menambahkan augmentasi PhotoMetricDistortion ke pipeline training. ---")

# Terapkan konfigurasi ke semua dataloader
cfg.train_dataloader.dataset.update(
    data_root=data_root, metainfo=metainfo, pipeline=train_pipeline,
    ann_file='train/coco.json', data_prefix=dict(img='train/'))
cfg.val_dataloader.dataset.update(
    data_root=data_root, metainfo=metainfo, pipeline=val_pipeline,
    ann_file='valid/coco.json', data_prefix=dict(img='valid/'))
cfg.test_dataloader.dataset.update(
    data_root=data_root, metainfo=metainfo, pipeline=val_pipeline,
    ann_file='test/coco.json', data_prefix=dict(img='test/'))

--- INFO: Melakukan fine-tuning untuk 17 kelas. ---
--- INFO: Menambahkan augmentasi PhotoMetricDistortion ke pipeline training. ---


In [ ]:
# ===== Step 4: Evaluator =====
cfg.val_evaluator.metric = 'bbox'
cfg.val_evaluator.ann_file = data_root + 'valid/coco.json'
cfg.test_evaluator.metric = 'bbox'
cfg.test_evaluator.ann_file = data_root + 'test/coco.json'

In [5]:

# ===== Step 5: Atur Kepala Model Deteksi (Model Head) =====
# Untuk RetinaNet, head-nya berbeda dari Faster R-CNN
# cfg.model.bbox_head.num_classes = NUM_CLASSES


if hasattr(cfg.model, 'bbox_head') and not hasattr(cfg.model, 'roi_head'):
    # Ini untuk arsitektur One-Stage seperti RetinaNet
    print(f"--- INFO: Menyetel bbox_head.num_classes ke {NUM_CLASSES} untuk RetinaNet. ---")
    cfg.model.bbox_head.num_classes = NUM_CLASSES
elif hasattr(cfg.model, 'roi_head'):
    # Ini untuk arsitektur Two-Stage seperti Faster R-CNN
    print(f"--- INFO: Menyetel roi_head.bbox_head.num_classes ke {NUM_CLASSES} untuk Faster R-CNN. ---")
    if isinstance(cfg.model.roi_head.bbox_head, list):
        # Untuk model dengan beberapa bbox_head (seperti Cascade R-CNN)
        for head in cfg.model.roi_head.bbox_head:
            head.num_classes = NUM_CLASSES
    else:
        # Untuk Faster R-CNN standar
        cfg.model.roi_head.bbox_head.num_classes = NUM_CLASSES


--- INFO: Menyetel roi_head.bbox_head.num_classes ke 17 untuk Faster R-CNN. ---


In [ ]:
#code untuk retinanet

# ===== Step 6: MELANJUTKAN Training yang Terputus =====

# # [PENTING] Path ke checkpoint terakhir dari training Tahap 2 yang crash
# checkpoint_terakhir = './outputs_swin_retinanet_ornamental_finetuned_advanced/epoch_38.pth'
# cfg.load_from = checkpoint_terakhir

# # [KUNCI UTAMA] Aktifkan mode resume
# # Ini akan memuat state optimizer, scheduler, dan nomor epoch, lalu melanjutkan
# cfg.resume = True

# print(f"--- INFO: Melanjutkan (RESUME) training dari: {checkpoint_terakhir} ---")


#code untuk faster-rcnn

# ===== Step 6: Pengaturan Checkpoint & Resume yang Benar =====

# Definisikan path ke folder kerja dan checkpoint yang diharapkan
#work dir untuk retinanet swin'
# WORK_DIR = './outputs_swin_retinanet_ornamental_finetuned_advanced'

#work_dir untuk swin faster rcnn
WORK_DIR = f'./outputs_swin_{model_name}_ornamental_finetuned_advanced'
latest_checkpoint_path = os.path.join(WORK_DIR, 'latest.pth')

# Logika utama
if os.path.exists(latest_checkpoint_path):
    # --- SKENARIO 1: Checkpoint DITEMUKAN, LANJUTKAN TRAINING ---
    print(f"--- INFO: Checkpoint terakhir ditemukan di {latest_checkpoint_path}. ---")
    print("--- INFO: Akan melanjutkan (RESUME) training. ---")
    cfg.load_from = latest_checkpoint_path  # Tentukan path checkpoint
    cfg.resume = True                       # AKTIFKAN mode resume
else:
    # --- SKENARIO 2: TIDAK ADA CHECKPOINT, MULAI DARI AWAL (PRE-TRAINED) ---
    print(f"--- INFO: Tidak ada checkpoint di {WORK_DIR}. ---")
    print("--- INFO: Memulai training baru (fine-tuning) dari bobot pre-trained Tahap 1. ---")
    
    # Ganti dengan path checkpoint dari TAHAP 1 (pre-training di DeepFish)
    tahap_1_checkpoint = './outputs_swin_retinanet_deepfish/best_coco_bbox_mAP_epoch_X.pth' # <-- GANTI INI
    
    if os.path.exists(tahap_1_checkpoint):
        cfg.load_from = tahap_1_checkpoint
    else:
        # Jika checkpoint tahap 1 juga tidak ada, bisa set ke None atau URL Swin asli
        print(f"PERINGATAN: Checkpoint Tahap 1 di {tahap_1_checkpoint} tidak ditemukan. Memuat dari bobot Swin-COCO asli.")
        # Biarkan cfg.load_from default dari file config (bobot pre-trained di COCO)
    
    cfg.resume = False # PASTIKAN resume false untuk training baru

--- INFO: Tidak ada checkpoint di ./outputs_swin_Faster-rcnn_ornamental_finetuned_advanced. ---
--- INFO: Memulai training baru (fine-tuning) dari bobot pre-trained Tahap 1. ---
PERINGATAN: Checkpoint Tahap 1 di ./outputs_swin_retinanet_deepfish/best_coco_bbox_mAP_epoch_X.pth tidak ditemukan. Memuat dari bobot Swin-COCO asli.


In [ ]:
# ===== Step 7: Konfigurasi Training (Fine-tuning Lanjutan) =====
cfg.optim_wrapper.optimizer.lr = 0.00001  # Mulai dengan LR yang terbukti baik
cfg.train_cfg.max_epochs = 50  # Tingkatkan total epoch
print(f"--- INFO: Training diatur untuk {cfg.train_cfg.max_epochs} epoch. ---")

# ===== [PENINGKATAN 3] Jadwal Training yang Lebih Panjang =====
# Gunakan scheduler yang menurunkan LR di tahap akhir untuk penyempurnaan
cfg.param_scheduler = [
    dict(type='LinearLR', start_factor=0.001, by_epoch=False, begin=0, end=500), # Warmup
    dict(
        type='MultiStepLR',
        by_epoch=True,
        begin=0,
        end=50,
        milestones=[35, 45], # Turunkan LR di epoch 35 dan 45
        gamma=0.1)
]
print("--- INFO: Menggunakan MultiStepLR scheduler, LR akan turun di epoch 35 & 45. ---")


cfg.default_hooks.logger.interval = 10
cfg.default_hooks.checkpoint.interval = 2 # Simpan setiap 2 epoch
cfg.default_hooks.checkpoint.max_keep_ckpts = 3
cfg.default_hooks.checkpoint.save_best = 'coco/bbox_mAP'
cfg.default_hooks.checkpoint.rule = 'greater'
cfg.visualizer.vis_backends = [dict(type='LocalVisBackend'), dict(type='TensorboardVisBackend')]

--- INFO: Training diatur untuk 50 epoch. ---
--- INFO: Menggunakan MultiStepLR scheduler, LR akan turun di epoch 35 & 45. ---


In [ ]:
# ===== Step 8: Output Folder Baru =====
# Gunakan nama folder baru untuk eksperimen ini
cfg.work_dir = WORK_DIR
os.makedirs(cfg.work_dir, exist_ok=True)

In [9]:
# ===== Step 9: Verifikasi Dataset =====
from mmengine.registry import DATASETS
train_dataset = DATASETS.build(cfg.train_dataloader.dataset)
print(f"\nJumlah data fine-tuning: {len(train_dataset)}")
print(f"Kelas yang akan dilatih: {train_dataset.metainfo['classes']}\n")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Jumlah data fine-tuning: 2440
Kelas yang akan dilatih: ('Fish', 'Angel Fish', 'Betta Candy Koi Fighting Fish', 'Candy Koi Betta', 'Galaxy Betta', 'Galaxy Betta Fish', 'Gemrin Koi', 'Guppy Dumbo', 'Guppy Dumbo Fish', 'Juvenile Jewel', 'Juvenile Jewel Fish', 'Koi Gemrin', 'Marble Molly', 'Marble Molly Fish', 'Polar Blue Parrot', 'Rancho', 'Tiger Barb')



In [10]:
# ===== Step 10: Mulai Fine-tuning =====
runner = Runner.from_cfg(cfg)
runner.train()
print(">>> Proses fine-tuning selesai.")

08/13 01:24:40 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1393167610
    GPU 0: NVIDIA GeForce RTX 5060
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
    NVCC: Cuda compilation tools, release 12.8, V12.8.61
    MSVC: n/a, reason: fileno
    PyTorch: 2.8.0+cu128
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 193833145
  - Intel(R) oneAPI Math Kernel Library Version 2025.2-Product Build 20250620 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.7.1 (Git Hash 8d263e693366ef8db40acc569cc7d8edf644556d)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.8
  - NVCC architecture flags: -genco

KeyboardInterrupt: 

In [ ]:
# ===== Step 11: Inference Otomatis + Rekap Deteksi =====
print("\n>>> Memulai inference pada gambar tes...")
from mmdet.apis import inference_detector, init_detector
from mmdet.visualization import DetLocalVisualizer
import mmcv
import numpy as np
from collections import Counter

# Cari checkpoint terbaik
best_ckpts = [f for f in os.listdir(cfg.work_dir) if f.startswith('best_') and f.endswith('.pth')]
if best_ckpts:
    checkpoint_path = os.path.join(cfg.work_dir, sorted(best_ckpts)[-1])
else:
    checkpoint_path = os.path.join(cfg.work_dir, "latest.pth")

print(f"--- INFO: Menggunakan checkpoint: {checkpoint_path} ---")

if os.path.exists(checkpoint_path):
    model = init_detector(cfg, checkpoint_path, device='cuda' if torch.cuda.is_available() else 'cpu')
    visualizer = DetLocalVisualizer(vis_backends=[dict(type='LocalVisBackend')])
    visualizer.dataset_meta = cfg.train_dataloader.dataset.metainfo

    img_path = 'dataset/Ornamental Freshwater Fish/test/IMG_0194_jpg.rf.2da382821c1042b160f37a96548228de.jpg'
    output_folder = os.path.join(cfg.work_dir, 'test_outputs')
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, 'result_image.jpg')

    if os.path.exists(img_path):
        # Jalankan inference
        result = inference_detector(model, img_path)

        # Ambil nama kelas
        class_names = cfg.train_dataloader.dataset.metainfo['classes']

        # Ambil prediksi bbox dari result
        pred_instances = result.pred_instances
        labels = pred_instances.labels.cpu().numpy()
        scores = pred_instances.scores.cpu().numpy()

        # Terapkan threshold skor
        score_thr = 0.4
        valid_idx = scores >= score_thr
        labels = labels[valid_idx]

        # Hitung jumlah per kelas
        counts = Counter(labels)

        # Simpan gambar hasil deteksi
        img = mmcv.imread(img_path)
        img = mmcv.imconvert(img, 'bgr', 'rgb')
        visualizer.add_datasample(
            name='Hasil Deteksi',
            image=img,
            data_sample=result,
            draw_gt=False,
            show=False,
            pred_score_thr=score_thr,
            out_file=output_path
        )
        print(f">>> Inference selesai. Hasil tersimpan di {output_path}")

        # Cetak rekap
        print("\n==== Summary Deteksi ====")
        if counts:
            for cls_id, count in counts.items():
                print(f"{class_names[cls_id]} = {count}")
        else:
            print("Tidak ada ikan terdeteksi.")
    else:
        print(f"ERROR: Gambar uji tidak ditemukan di {img_path}")
else:
    print("ERROR: Tidak ada checkpoint untuk inference.")



>>> Memulai inference pada gambar tes...


NameError: name 'os' is not defined